In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 18 #字体大小
plt.rcParams['font.family'] = ['Arial Unicode MS'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
%matplotlib inline

import featuretools as ft
print('ft.__version__:\t',ft.__version__)

ft.__version__:	 0.23.0


In [2]:
dt_file =  'train/data/dt/'

In [3]:
# 加载数据
data = pd.read_excel(dt_file+'data_new.xlsx')
data.sample(3)

,balance_capital,FINANCE_PRODUCT_CODE,name,curr_overdue,id_number,OVERDUE_DAYS,apply_date,max_overdue,LOAN_DATE,target,...,br.按身份证号查询，近12个月最大月申请次数,zzc.D90.身份证号码.贷款类型列表_len,br.按手机号查询，近3个月在非银机构-消费类分期申请机构数,zzc.D720.手机号码.机构类型列表_len,zzc.D90.身份证号码.其他类.申请数,br.ir_allmatch_days,td.zrf.智融分2.0银行分标准版,br.按手机号查询，近12个月申请线下消费分期的机构数,zzc.D540.手机号码.平均换新机构间隔时长,zzc.D540.身份证号码.贷款类型列表_len
14191,624.0,PDL02,d3d7999bfe1333fd55b8ae75893c06ff,0.0,dde21e0b60c727da3cf5205f3d7c29f3,-1,2020-04-02,0.0,02APR2020,0,...,17.0,27,5.0,55,5.0,1.0,490,3.0,55.0,34
8292,NaN,PDL01hc,5e6eeee2b0ce642e286073ef8d4f8f47,NaN,ae28b50b2d55e657bd5c0f32ce007ecf,0,2020-04-02,NaN,01MAY2020,-999,...,NaN,0,NaN,19,0.0,103.0,704,3.0,-1.0,16
5212,0.0,PDL02,63040d9b963fa6e776492e40b422584a,0.0,3e4c5b335d6393d1e559d7deffc6cd36,-1,2020-03-08,0.0,08MAR2020,0,...,5.0,11,4.0,11,0.0,1.0,659,1.0,0.0,11


In [160]:
#------------------------手动删除特征--------------------
# 加载特征
with open(dt_file+'feature_new.txt','r') as f:
    features = f.read().split('\n')
    
with open(dt_file+'cat_feature_new.txt','r') as f:
    cat_features = f.read().split('\n')

self_drop = ['mob.fraud_stage'
            ]

features = [i for i in features if i not in self_drop]
cat_features = [i for i in cat_features if i not in self_drop]
print(len(features))
print(cat_features)

153
['zzc.D180.身份证号码.贷款类型列表.consumerfinance']


In [163]:
import copy
dt = copy.deepcopy(data)
train = dt.sample(1000)

In [164]:
# 创建实体集
es_train = ft.EntitySet(id='train')
es_data = ft.EntitySet(id='predict')

# 添加实体
df_config = {
    'entity_id':'data',
    'index':'C_APP_ID',
    'variable_types':{
        **{col:ft.variable_types.Categorical for col in cat_features},
        **{col:ft.variable_types.Numeric for col in features if col not in cat_features}
    },
    'make_index':False,
    'time_index':None
}

es_train = es_train.entity_from_dataframe(dataframe = train,**df_config)
es_data = es_data.entity_from_dataframe(dataframe = dt,**df_config)

In [170]:
# # Running DFS
import re
# # 计算基元
trans_primitives = ['divide_numeric']

ignore_str = """比例|score|days|识别|天数|评分|风险识别|SCORE|GENDER|
AGE|td\.zrf|标准差|环比|len|时长|列表|间隔|addr_cnt|手机号数|设备号的个数|其他|N_AGE|事件中的记录条数|
贷款事件中发生在白天的次数|P2P"""

ignore_variables = [col for col in dt.columns if col not in features or bool(re.search(ignore_str,col))]
# # matrix_dfs,
features_dfs = ft.dfs(entityset=es_train,
                                    target_entity='data',
                                    ignore_variables = {'data':ignore_variables},
                                    trans_primitives=trans_primitives,
                                    max_depth=2,
                                    n_jobs= -1,
                                    verbose=False,
                                    features_only=True)

In [169]:
# set(dt.columns) - set(ignore_variables)

In [171]:
len(features_dfs)

6724

In [172]:
features_dfs[-10:]

[<Feature: zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的7天内全行业下贷款事件中的不同平台个数>,
 <Feature: zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的90天内全行业下贷款事件中的不同平台个数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D180.身份证号码.银行类.申请数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D30.手机号码.总申请数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.总申请机构数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请机构数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D540.身份证号码.总申请数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D720.手机号码.非银行持牌类.申请数>,
 <Feature: zzc.D90.身份证号码.总申请数 / zzc.D720.身份证号码.银行类.申请数>]

In [173]:
matrix_dfs = ft.calculate_feature_matrix(features_dfs, es_data)
print(matrix_dfs.shape)
matrix_dfs.sample(5)

(14910, 6724)


,zzc.D360.身份证号码.总申请机构数,br.按手机号查询，近3个月申请线上小额现金贷的机构数,br.按手机号查询，近3个月在非银机构-现金类分期机构申请次数,br.按身份证号查询，近6个月在非银机构有申请记录月份数,br.按手机号查询，近12个月有申请记录月份数,br.按身份证号查询，近3个月最小月申请次数,br.按身份证号查询，近3个月申请线下现金分期的次数,br.按手机号查询，近15天在非银机构申请次数,br.按手机号查询，近6个月申请线上小额现金贷的机构数,br.按手机号查询，近12个月申请线下消费分期的次数,...,zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的7天内全行业下贷款事件中的不同平台个数,zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的90天内全行业下贷款事件中的不同平台个数,zzc.D90.身份证号码.总申请数 / zzc.D180.身份证号码.银行类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D30.手机号码.总申请数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.总申请机构数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请机构数,zzc.D90.身份证号码.总申请数 / zzc.D540.身份证号码.总申请数,zzc.D90.身份证号码.总申请数 / zzc.D720.手机号码.非银行持牌类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D720.身份证号码.银行类.申请数
C_APP_ID,,,,,,,,,,,,,,,,,,,,,
2489550556200978,1.0,1.0,NaN,1.0,7.0,1.0,NaN,NaN,1.0,NaN,...,inf,0.500000,inf,inf,1.000000,0.333333,1.0,0.250000,1.000000,inf
2449715434889277,5.0,2.0,1.0,5.0,7.0,2.0,NaN,2.0,2.0,1.0,...,4.0,0.444444,4.00,4.0,0.800000,1.333333,2.0,0.666667,4.000000,4.000
2447558749061142,17.0,2.0,NaN,6.0,12.0,5.0,2.0,2.0,5.0,16.0,...,7.0,0.500000,1.75,7.0,0.411765,1.166667,1.4,0.170732,1.166667,0.875
2461193055502372,4.0,1.0,NaN,3.0,5.0,0.0,1.0,3.0,1.0,5.0,...,0.0,0.000000,NaN,NaN,0.000000,0.000000,0.0,0.000000,0.000000,NaN
2416005800747022,0.0,1.0,NaN,3.0,7.0,0.0,NaN,NaN,1.0,NaN,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


In [174]:
matrix_dfs.reset_index(drop=False,inplace=True)
print(matrix_dfs.shape)
matrix_dfs.sample(2)

(14910, 6725)


,C_APP_ID,zzc.D360.身份证号码.总申请机构数,br.按手机号查询，近3个月申请线上小额现金贷的机构数,br.按手机号查询，近3个月在非银机构-现金类分期机构申请次数,br.按身份证号查询，近6个月在非银机构有申请记录月份数,br.按手机号查询，近12个月有申请记录月份数,br.按身份证号查询，近3个月最小月申请次数,br.按身份证号查询，近3个月申请线下现金分期的次数,br.按手机号查询，近15天在非银机构申请次数,br.按手机号查询，近6个月申请线上小额现金贷的机构数,...,zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的7天内全行业下贷款事件中的不同平台个数,zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的90天内全行业下贷款事件中的不同平台个数,zzc.D90.身份证号码.总申请数 / zzc.D180.身份证号码.银行类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D30.手机号码.总申请数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.总申请机构数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请机构数,zzc.D90.身份证号码.总申请数 / zzc.D540.身份证号码.总申请数,zzc.D90.身份证号码.总申请数 / zzc.D720.手机号码.非银行持牌类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D720.身份证号码.银行类.申请数
250,2473968651091972,12.0,5.0,NaN,6.0,12.0,7.0,3.0,2.0,6.0,...,inf,0.388889,7.0,inf,0.583333,7.0,7.0,0.304348,1.4,3.5
8394,2462647600660544,5.0,1.0,NaN,2.0,8.0,0.0,NaN,NaN,1.0,...,NaN,0.000000,NaN,NaN,0.000000,0.0,0.0,0.000000,0.0,NaN


In [175]:
ignore_dt = copy.deepcopy(dt[ignore_variables])
ignore_dt.sample(2)

,balance_capital,FINANCE_PRODUCT_CODE,name,curr_overdue,id_number,OVERDUE_DAYS,apply_date,max_overdue,LOAN_DATE,target,...,mob.wd_score_v2,td.zrf.小额多期分标准版_,td.zrf.智融分2.0非银分标准版,zzc.D90.身份证号码.贷款类型列表_len,zzc.D720.手机号码.机构类型列表_len,zzc.D90.身份证号码.其他类.申请数,br.ir_allmatch_days,td.zrf.智融分2.0银行分标准版,zzc.D540.手机号码.平均换新机构间隔时长,zzc.D540.身份证号码.贷款类型列表_len
3497,2364.0,PDL02,970ef3e829829157288f1e355f977b47,0.0,e96386d8b379979406013892f16945b8,0,2020-05-05,4.0,2020-05-27,0,...,670,674,601,16,34,0.0,3.0,579,425.0,36
5507,1234.0,PDL02,7337f864b0ae01edf292aae6095388d0,0.0,e490c82355c2e7fe74af8841b0ee9ac9,2,2020-05-07,1.0,2020-05-07,0,...,765,657,689,19,51,1.0,14.0,568,54.0,47


In [176]:
for col in ignore_dt.columns:
    if col in matrix_dfs.columns:
        print(col)

C_APP_ID


In [177]:
matrix_dfs_new = pd.merge(ignore_dt,matrix_dfs,on='C_APP_ID')
matrix_dfs_new.head(2)

,balance_capital,FINANCE_PRODUCT_CODE,name,curr_overdue,id_number,OVERDUE_DAYS,apply_date,max_overdue,LOAN_DATE,target,...,zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的7天内全行业下贷款事件中的不同平台个数,zzc.D90.身份证号码.总申请数 / td.dqsh.身份证关联到的90天内全行业下贷款事件中的不同平台个数,zzc.D90.身份证号码.总申请数 / zzc.D180.身份证号码.银行类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D30.手机号码.总申请数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.总申请机构数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D360.身份证号码.非银行持牌类.申请机构数,zzc.D90.身份证号码.总申请数 / zzc.D540.身份证号码.总申请数,zzc.D90.身份证号码.总申请数 / zzc.D720.手机号码.非银行持牌类.申请数,zzc.D90.身份证号码.总申请数 / zzc.D720.身份证号码.银行类.申请数
0,0.0,PDL02,affb82d901b4220d6d81abf710763c49,0.0,4370ba14897480c70b37a239dc14db52,-10,2020-03-11,0.0,2020-03-11,0,...,1.0,0.090909,inf,inf,1.0,1.0,1.0,1.0,1.0,inf
1,499.0,PDL02,8b76a1d59368bb1259ceb14903463b23,0.0,711b52b23ebb745e7e806ad619d21cdd,40,2020-03-29,39.0,2020-04-13,1,...,1.0,0.200000,1.0,1.0,0.2,inf,inf,0.2,inf,0.5


In [179]:
len(ignore_variables)

88

In [180]:
label_cols = ['C_APP_ID','APP_CHANNEL','id_number','name','mobile','apply_date','LOAN_DATE','FINANCE_PRODUCT_CODE',
             'TENOR','OVERDUE_DAYS','target_m2','target','balance_capital','curr_overdue','max_overdue','f6_max_overdue',
             'loan_amt']

In [188]:
features_new = list((set([f.get_name() for f in features_dfs])|set(ignore_variables)) - set(label_cols))
for col in features_new:
    if col not in matrix_dfs_new.columns:
        print(col)
len(features_new)

6795

In [189]:
matrix_dfs_new['target'].value_counts()

 0      11480
 1       1561
-999     1223
 2        646
Name: target, dtype: int64

In [190]:
# 保存dfs_features
ft.save_features(features_dfs, f'{dt_file}dfs_features_tmp.json')
# 保存数据
matrix_dfs_new.to_csv(f'{dt_file}data_dfs_tmp.csv',index=False)

In [191]:
# 加载
features_dfs = ft.load_features(f'{dt_file}dfs_features_tmp.json')
matrix_dfs_new = pd.read_csv(f'{dt_file}data_dfs_tmp.csv')

In [192]:
matrix_dfs_new_1 = matrix_dfs_new.loc[(matrix_dfs_new['target'] != -999)&(matrix_dfs_new['target'] != 2),:]
matrix_dfs_new_1.shape

(13041, 6812)

In [193]:
# 删除不同类型交叉变量
drop_col = []
for f in features_new:
    if r'/' in f:
        t = f.split(r'/')
        if t[0].split(r'.')[0].strip() != t[1].split(r'.')[0].strip():
            drop_col.append(f)
len(drop_col)

2466

In [194]:
features_new = list(set(features_new)-set(drop_col))
len(features_new)

4329

In [196]:
IV_dict = {}
for i,col in enumerate(features_new):
    if i%500==0:
        print(i)
    IV_dict[col] = get_IV(matrix_dfs_new_1[col],matrix_dfs_new_1['target'])

0
500
1000
1500
2000
2500
3000
3500
4000


In [197]:
features_new = [f for f in features_new if IV_dict[f] >= 0.02]
len(features_new)

555

In [198]:
f_tmp_list = copy.deepcopy(features_new)
for f in f_tmp_list:
    if '/' in f:
        if (IV_dict[f] <= IV_dict[f.split('/')[0].strip()]+0.01) \
           or (IV_dict[f] <= IV_dict[f.split('/')[1].strip()]+0.01):
            try:
                features_new.remove(f)
            except:
                print(f)
len(features_new)

139

In [199]:
# 保存IV
import joblib
joblib.dump({k:v for k,v in IV_dict.items() if v >= 0.02},'./result/dfs/feature_dfs_iv.pkl')

['./result/dfs/feature_dfs_iv.pkl']

In [200]:
na = NanDeal(df=matrix_dfs_new_1,cols=features_new)
missCount = na.miss_count
features_new = [f for f in features_new if missCount[f] < 0.6]
len(features_new)

136

In [201]:
f_tmp_list = copy.deepcopy(features_new)
drop_cols = []
for f in f_tmp_list:
    if '/' in f:
        try:
            if (missCount[f] >= missCount[f.split('/')[0].strip()]) \
            or (missCount[f] >= missCount[f.split('/')[1].strip()]):
                drop_cols.append(f)
        except KeyError:
            pass
features_new = list(set(features_new)-set(drop_cols))
len(features_new)

128

In [215]:
md = ModeDeal(df = matrix_dfs_new_1, cols = features_new)
modeCount = md.mode_values
features_new = [f for f in features_new if modeCount[f] < 0.6]
len(features_new)

{'br.按手机号查询，近15天在非银机构申请次数': 0.20128824476650564,
 'br.按手机号查询，近3个月在非银机构-持牌网络小贷机构申请机构数': 0.2221455409861207,
 'td.dqsh.身份证关联到的270天内非银行类金融行业下申贷事件中近90日的新增平台数': 0.20933977455716588}

In [203]:
f_tmp_list = copy.deepcopy(features_new)
drop_cols = []
for f in f_tmp_list:
    if '/' in f:
        try:
            if (modeCount[f] >= modeCount[f.split('/')[0].strip()]) \
            or (modeCount[f] >= modeCount[f.split('/')[1].strip()]):
                drop_cols.append(f)
        except KeyError:
            pass
features_new = list(set(features_new)-set(drop_cols))
len(features_new)

128

In [143]:
# 特征选择
# import toad
# matrix_dfs_new_1, drop_cols = toad.selection.select(frame = matrix_dfs_new_1,
#                                                   target = matrix_dfs_new_1['target'],
#                                                 empty=0.6,
#                                                 iv=0.02,
#                                                 corr=0.7,
#                                                 return_drop=True,
#                                                 exclude = label_cols)
# print(matrix_dfs_new_1.shape)
# matrix_dfs_new_1.sample(4)

In [144]:
# features_new = list(set(features_new)&set(matrix_dfs_new_1.columns))
# len(features_new)

In [204]:
features_dfs_new = [i for i in features_dfs if i.get_name() in features_new]
len(features_dfs_new)

110

In [205]:
# 保存dfs_features,用于部署
ft.save_features(features_dfs_new, f'{dt_file}dfs_features.json')

In [206]:
matrix_dfs_new[label_cols+features_new].sample(2)

,C_APP_ID,APP_CHANNEL,id_number,name,mobile,apply_date,LOAN_DATE,FINANCE_PRODUCT_CODE,TENOR,OVERDUE_DAYS,...,br.按手机号查询，近12个月申请线下消费分期的次数 / br.按身份证号查询，近6个月在非银机构申请机构数,br.按身份证号查询，近3个月在非银机构申请最大间隔天数,br.按手机号查询，近12个月申请线上现金分期的次数 / br.按身份证号查询，近3个月在非银机构-持牌消费金融机构申请机构数,br.按手机号查询，近12个月申请线上现金分期的次数 / br.按手机号查询，近1个月在非银机构-持牌网络小贷机构申请机构数,br.按身份证号查询，近3个月在非银机构-消费类分期机构申请次数 / br.按手机号查询，近12个月申请线上现金分期的次数,br.按手机号查询，近12个月申请线上现金分期的次数 / br.按手机号查询，近12个月在非银机构-持牌网络小贷机构申请机构数,br.按手机号查询，近12个月申请线下消费分期的次数 / br.按手机号查询，近12个月在非银机构-持牌消费金融机构申请机构数,br.按身份证号查询，近12个月最大月申请次数,br.按身份证号查询，近3个月申请线上现金分期的次数 / br.按手机号查询，近3个月在非银机构-持牌网络小贷机构申请机构数,br.按身份证号查询，近3个月申请线上小额现金贷的次数 / br.按手机号查询，近12个月申请线下消费分期的次数
3763,2415069744234541,RONGSHU,31eaea0c55d54871a136ab053e5271a9,947e08850ca19f09de1461d11b1d9b2a,8d9fb3ea2ecf8b6f9bfefe6e41fa98a8,2020-03-08,2020-03-08,PDL02,12,5,...,NaN,17.0,3.0,NaN,0.666667,1.500000,NaN,3.0,NaN,NaN
12355,2471213474644065,PINTAI01,d38df08c4aa1fe4be13e95b50c0fc975,4b4b8f36ace9c33975d898bdb2a845ca,1d82ae48c0942bc16786b02d67261a04,2020-05-26,2020-05-26,PDL02,12,-1,...,0.428571,24.0,20.0,NaN,0.400000,2.857143,0.5,11.0,4.0,1.666667


In [207]:
# 保存数据
matrix_dfs_new[label_cols+features_new].to_csv(f'{dt_file}data_dfs.csv',index=False)

In [208]:
# 保存变量
with open(f'{dt_file}features_dfs.txt','w') as f:
    f.write('\n'.join(features_new))

In [209]:
IV_dict[features_new[0]]

0.032211361321928256

In [210]:
# 输出衍生变量逻辑图
for f in features_dfs_new:
    if  '/' in f.get_name():
        if IV_dict[f.get_name()] >= 0.03 \
        and IV_dict[f.get_name().split('/')[0].strip()] < IV_dict[f.get_name()] \
        and IV_dict[f.get_name().split('/')[1].strip()] < IV_dict[f.get_name()]:
            ft.graph_feature(feature = f,to_file=f'./result/dfs/{f.get_name()}.png',description=True)

In [152]:
from collections import Counter
class ModeDeal(object):
    """集中度处理"""
    def __init__(self,df:pd.DataFrame,cols:list=None):
        self.df = copy.deepcopy(df)
        if not cols:
            self.cols = df.columns.tolist()
        else:
            self.cols = copy.deepcopy(cols)
        self.mode_values = self.mode_count()

    def mode_count(self):
        mode_values = self.df[self.cols].mode(dropna=True).iloc[0].to_dict()
        mode_ratio = {k:Counter(self.df[k]).get(v)/len(self.df)
                      for k,v in mode_values.items()}
        return mode_ratio

    def drop_mode(self,keep_cols:list=None,return_drop:bool=True,mode:float=0.8):
        if not keep_cols:
            keep_cols = []
        drop_cols = [k for k in self.cols if self.mode_values.get(k) >= mode and k not in keep_cols]
        self.df = self.df[self.df.columns.difference(drop_cols)]
        if return_drop:
            return self.df, drop_cols
        else:
            return self.df

In [153]:
import copy
class NanDeal(object):
    """缺失处理"""
    def __init__(self, df: pd.DataFrame, cols: list = None):
        self.df = copy.deepcopy(df)
        if not cols:
            self.cols = df.columns.tolist()
        else:
            self.cols = copy.deepcopy(cols)
        self.miss_count = self.nan_count()

    def nan_count(self):
        result = (self.df[self.cols].isnull().sum() / len(self.df)).to_dict()
        return result

    def fill_nan(self, spvalue: [str, int, float, dict] = -9999, limit_ratio=0.05):
        less_miss = [k for k, v in self.miss_count.items() if v < limit_ratio]
        # 缺失较低
        for col in less_miss:
            self.df[col] = self.df[col].fillna(self.df[col].mode(dropna=True)[0])
            if self.df[col].isnull().values.any():
                self.df[col] = self.df[col].fillna('nan')
        # 指定填充
        if isinstance(spvalue, dict):
            spvalue = {k: {np.nan: v} for k, v in spvalue.items()}

        else:
            spvalue = {np.nan: spvalue}

        self.df.replace(to_replace=spvalue, inplace=True)

        return self.df

In [195]:
def get_IV(x:pd.Series, y:pd.Series):
    # 计算特征IV
    def goodBad(df):
        names = {'good': (df['y']==0).sum(),'bad': (df['y']==1).sum()}
        return pd.Series(names)
    # 对x进行等频分
    if len(set(x)) > 20:
        q = 10
    elif len(set(x)) <= 10:
        q = len(set(x)) - 2
        if q < 0:
            q = 0
    else:
        q = 4

    x = pd.qcut(x=x,q=q,labels=False,duplicates='drop')
    iv_total = pd.DataFrame({'x':x,'y':y})\
        .fillna(-99)\
        .groupby('x')\
        .apply(goodBad)\
        .replace(0, 1)\
        .assign(
        DistrBad = lambda x: x.bad/sum(x.bad),DistrGood = lambda x: x.good/sum(x.good)
    ).\
        assign(iv = lambda x: (x.DistrBad-x.DistrGood)*np.log(x.DistrBad/x.DistrGood))\
        .iv.sum()
    # return iv
    return iv_total